table of contents

1. データの読み込み・ローカルへのダウンロード

## 必要な追加ライブラリのインストール
- google colabという環境には、データ分析や機械学習に必要な最低限のライブラリはインストール済
- 本notebookの目的であるイベントデータをダウンロードするためには、別途`statsbomb`と`kloppy`というライブラリをインストールする必要がある

In [1]:
!pip install statsbomb
!pip install kloppy

## ライブラリのインポート
- pythonで便利なライブラリを使うには、実行環境へインストールするだけでなく、importという作業が必要
- `import hogehoge as hoge`のように、別名で定義することもある

In [2]:
import requests
import math
import itertools

import numpy as np
import pandas as pd

from kloppy import datasets
import statsbomb as sb

定数の定義（処理中で書き換えられない情報は大文字で定義することが多い）

In [3]:
BASE_URL = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data'

# データの読み込み（[github/statsbomb/opden-data](https://github.com/statsbomb/open-data)から）、ダウンロード
- ここでは、上記リンクにあるjson形式のオープンデータを、人間が取り扱いやすい表形式（dataframe）に変換しながらダウンロードする
- ダウンロードする情報としては、「大会情報」「試合情報」「選手情報」「イベントデータ（実際に分析するもの、いつどこで誰がどんなプレイをしたか）」

## 大会情報
- 各列の情報は上手く読んでほしい
- CLについてはファイナルのみ、Ligaについてはバルセロナのみ。大会を通してみれるのはロシアWCがよい

In [4]:
comps_df = sb.Competitions().get_dataframe()

comps_df

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2020-07-29T05:00,2020-07-29T05:00
1,16,1,Europe,Champions League,male,2017/2018,2020-07-29T05:00,2020-07-29T05:00
2,16,2,Europe,Champions League,male,2016/2017,2020-08-26T12:33:15.869622,2020-07-29T05:00
3,16,27,Europe,Champions League,male,2015/2016,2020-08-26T12:33:15.869622,2020-07-29T05:00
4,16,26,Europe,Champions League,male,2014/2015,2020-08-26T12:33:15.869622,2020-07-29T05:00
5,16,25,Europe,Champions League,male,2013/2014,2020-08-26T12:33:15.869622,2020-07-29T05:00
6,16,24,Europe,Champions League,male,2012/2013,2020-08-26T12:33:15.869622,2020-07-29T05:00
7,16,23,Europe,Champions League,male,2011/2012,2020-08-26T12:33:15.869622,2020-07-29T05:00
8,16,22,Europe,Champions League,male,2010/2011,2020-07-29T05:00,2020-07-29T05:00
9,16,21,Europe,Champions League,male,2009/2010,2020-07-29T05:00,2020-07-29T05:00


## 試合情報

### 関数の定義

In [5]:
def get_matches_df(competition_id, season_id=None, comps_df=None):
    """試合情報を返すメソッド
        Args:
            - competition_id(int) : 大会id
            - season_id(int, default=None) : シーズンid
            - comps_df(pd.DataFrame, default=None) : 大会情報
        Returns:
            pd.DataFrame : 試合情報
    """
    # if文、season_idがあれば、シーズンを指定してjson形式のデータをdataframe形式に変換する
    # season_idが指定されなければ、大会情報からseason_idをfor loopで回しながらダウンロード、変換する
    if season_id:
        matches_df = pd.DataFrame(requests.get(f'{BASE_URL}/matches/{competition_id}/{season_id}.json').json())
    else:
        # リストの内包表記
        matches_df = pd.concat([pd.DataFrame(requests.get(f'{BASE_URL}/matches/{competition_id}/{season_id}.json').json()) for season_id in comps_df[comps_df.competition_id==competition_id].season_id.tolist()])
    
    # ここでのmatches_dfは、エクセルで言うセルの中に辞書形式（dict）で値が入っていて分析しづらいので、それらをカラムに分解する
    c_list = ['competition', 'season', 'home_team', 'away_team', 'stadium', 'competition_stage']
    if competition_id == 2:
        c_list.remove('stadium')
    for c in c_list:
        if c in ['stadium', 'competition_stage']:
            key_list = ['id', 'name']
            c_fixed_list = [f'{c}_{k}' for k in key_list]

        else:
            key_list = [f'{c}_{k}' for k in ['id', 'name']]
            c_fixed_list = key_list
            
        for k, c_fixed in zip(key_list, c_fixed_list):
            matches_df[c_fixed] = matches_df[c].apply(lambda x: x[k] if type(x)==dict else None)
        
    # 必要なカラムのみを残して最終形とする
    matches_df = matches_df.drop(c_list+['metadata','referee'], axis=1).sort_values('match_date').reset_index(drop=True)

    return matches_df

### 実行
- competition_id = 大会id、上の表から選択
- season_id = シーズンid、(必要であれば)

In [6]:
competition_id = 11 # Liga <-ここを自由に編集
season_id = 22 # 2010/11 <- ここも

get_matches_df(competition_id=competition_id, season_id=season_id, comps_df=comps_df)

,match_id,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,competition_id,competition_name,season_id,season_name,home_team_id,home_team_name,away_team_id,away_team_name,stadium_id,stadium_name,competition_stage_id,competition_stage_name
0,69286,2010-08-29,19:00:00.000,0,3,available,2020-07-29T05:00,1,11,La Liga,22,2010/2011,1217,Racing Santander,217,Barcelona,4636,Campos de Sport de El Sardinero,1,Regular Season
1,69287,2010-09-11,18:00:00.000,0,2,available,2020-07-29T05:00,2,11,La Liga,22,2010/2011,217,Barcelona,1218,Hércules,342,Camp Nou,1,Regular Season
2,69269,2010-09-19,19:00:00.000,1,2,available,2020-07-29T05:00,3,11,La Liga,22,2010/2011,212,Atlético Madrid,217,Barcelona,625,Estadio Vicente Calderón,1,Regular Season
3,69236,2010-10-03,19:00:00.000,1,1,available,2020-07-29T05:00,6,11,La Liga,22,2010/2011,217,Barcelona,1043,Mallorca,342,Camp Nou,1,Regular Season
4,69260,2010-10-16,20:00:00.000,2,1,available,2020-07-29T05:00,7,11,La Liga,22,2010/2011,217,Barcelona,207,Valencia,342,Camp Nou,1,Regular Season
5,69263,2010-10-23,18:00:00.000,0,2,available,2020-07-29T05:00,8,11,La Liga,22,2010/2011,395,Real Zaragoza,217,Barcelona,4457,Estadio de la Romareda,1,Regular Season
6,69289,2010-10-30,22:00:00.000,5,0,available,2020-07-29T05:00,9,11,La Liga,22,2010/2011,217,Barcelona,213,Sevilla,342,Camp Nou,1,Regular Season
7,69270,2010-11-07,19:00:00.000,1,3,available,2020-07-29T05:00,10,11,La Liga,22,2010/2011,216,Getafe,217,Barcelona,354,Coliseum Alfonso Pérez,1,Regular Season
8,69308,2010-11-13,22:00:00.000,3,1,available,2020-07-29T05:00,11,11,La Liga,22,2010/2011,217,Barcelona,222,Villarreal,342,Camp Nou,1,Regular Season
9,69306,2010-11-20,20:00:00.000,0,8,available,2020-07-29T05:00,12,11,La Liga,22,2010/2011,403,Almería,217,Barcelona,4657,Estadio de los Juegos Mediterráneos,1,Regular Season


## データセット
- match_id = 試合id（上記から見たい試合を選択）

In [7]:
match_id = 69299 # Real Madrid vs Barcelona <- ここを自由に編集

# datasetというインスタンスでmetadataやイベントデータなどを管理
# これが前述のkloopyというライブラリで提供されているクラスを一時的に、インスタンスという形で保持している
dataset = datasets.load(dataset_name='statsbomb', match_id=match_id)

metadata = dataset.metadata
home_team, away_team = metadata.teams

### 対戦チームの表示

In [8]:
print(f"{home_team.ground} - {home_team}")
print(f"{away_team.ground} - {away_team}")

home - Real Madrid
away - Barcelona


## ラインナップ（スターティングイレブン等の選手情報）

In [9]:
lineups_df = pd.concat(
    [
        pd.DataFrame(data=[
            {'player_id':player.player_id, 'player_name':player.name, 'jersey_no':player.jersey_no, 'starting_flg':player.starting, 'team_id':team.team_id, 'team_name':team.name} 
            for player in team.players]
        ) 
        for team in metadata.teams
    ], ignore_index=True
)

lineups_df

,player_id,player_name,jersey_no,starting_flg,team_id,team_name
0,2995,Ángel Fabián Di María Hernández,22,True,220,Real Madrid
1,3181,Lassana Diarra,10,False,220,Real Madrid
2,3496,Mesut Özil,23,True,220,Real Madrid
3,5201,Sergio Ramos García,4,True,220,Real Madrid
4,5207,Cristiano Ronaldo dos Santos Aveiro,7,True,220,Real Madrid
5,5552,Marcelo Vieira da Silva Júnior,12,True,220,Real Madrid
6,5566,Sami Khedira,24,True,220,Real Madrid
7,10805,Ricardo Alberto Silveira de Carvalho,2,True,220,Real Madrid
8,11179,Iker Casillas Fernández,1,True,220,Real Madrid
9,19677,Karim Benzema,9,True,220,Real Madrid


## イベントデータのダウンロード（これが分析するデータ）

In [10]:
from kloppy import to_pandas

# イベントデータへ、ラインナップデータをidでくっつける
# エクセルでいうvlookupのような形
df = pd.merge(to_pandas(dataset), lineups_df, on=['player_id', 'team_id'], how='left')

df.head()

,event_id,event_type,result,success,period_id,timestamp,end_timestamp,ball_state,ball_owning_team,team_id,...,coordinates_x,coordinates_y,end_coordinates_x,end_coordinates_y,receiver_player_id,set_piece_type,player_name,jersey_no,starting_flg,team_name
0,ac926dec-ffa1-44ea-a734-dfc0a565a6df,GENERIC:Starting XI,None,None,1,0.000,NaN,alive,217,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c9bda184-571e-4482-bf23-990874e40b0d,GENERIC:Starting XI,None,None,1,0.000,NaN,alive,217,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b3dae79e-facf-499b-a1cb-2b6c5c59c81c,GENERIC:Half Start,None,None,1,0.000,NaN,alive,217,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7493688e-145f-4458-bd76-549bc7f21bec,GENERIC:Half Start,None,None,1,0.000,NaN,alive,217,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,78b837e8-5477-46cf-a401-d4cae5895637,PASS,COMPLETE,True,1,0.724,1.4029,alive,220,220,...,60.95,40.95,61.35,42.95,3496,KICK_OFF,Karim Benzema,9.0,True,Real Madrid


### カラム情報の確認

In [11]:
df.columns

Index(['event_id', 'event_type', 'result', 'success', 'period_id', 'timestamp',
       'end_timestamp', 'ball_state', 'ball_owning_team', 'team_id',
       'player_id', 'coordinates_x', 'coordinates_y', 'end_coordinates_x',
       'end_coordinates_y', 'receiver_player_id', 'set_piece_type',
       'player_name', 'jersey_no', 'starting_flg', 'team_name'],
      dtype='object')

## データのダウンロード

In [ ]:
from google.colab import files

filename = f'{home_team.name}vs{away_team.name}.csv'

df.to_csv(filename, index=False)

files.download(filename)